Import system dependencies

In [1]:
import sys
import os
sys.path.append('../')
import shutil
import statistics
import numpy as np

Import custom packages

In [2]:
from Utils import shuffleDataset, SplitJPData, saveToPickle, loadFromPickle, SplitChopinData, SplitMozartData, SplitBachData
from JPDataPreProcessing import toVectorTrainFormat, toVectorTestFormat, toInterleavedTrainFormat, toVectorFutureTrainFormat
from TrainModel import trainModel
from TestVectorModel import loadModel, testVecModelSave, testVecModelEval
from EvaluateJPMethod import getScoresForDL, getScoresForHmm

from ReferenceHMM import GetESTFingering
from parameters import DATA_DIR, HMM_RES_DIR

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


Split Dataset According to Composer

In [3]:
train_files, test_files, hmm_res_files = SplitBachData(DATA_DIR)
# train_files, test_files, hmm_res_files = SplitMozartData(DATA_DIR)
# train_files, test_files, hmm_res_files = SplitBachData(DATA_DIR)

Train all 3 model of reference HMM with the data splited above

In [4]:
os.chdir('../ReferenceHMM')
GetESTFingering.prepareInputList(train_files)
GetESTFingering.trainHmm()

Run trained HMM model, change GetESTFingering.FHMMx to swith models

In [5]:
test_filenames = GetESTFingering.getFormattedTestFilenames(test_files)
GetESTFingering.runHmm(test_filenames, GetESTFingering.FHMM1, default=False)
GetESTFingering.convertToCsv(input_dir='ESTResults/selfTrained/'+GetESTFingering.FHMM1+'/', output_dir='JPESTResults/selfTrained/'+GetESTFingering.FHMM1+'/')

move the files the to correct folders

In [6]:
if os.path.exists(HMM_RES_DIR + GetESTFingering.FHMM1):
    shutil.rmtree(HMM_RES_DIR + GetESTFingering.FHMM1)
shutil.move('JPESTResults/selfTrained/' + GetESTFingering.FHMM1, HMM_RES_DIR)
os.chdir('../LSTM Approach')

Evaluate hmm model

In [7]:
getScoresForHmm(test_files, hmm_res_files, GetESTFingering.FHMM1)

Total mean: 
M_GEN:  0.5724991389949875
M_HIGH:  0.6403457066119016
M_SOFT:  0.8447943944896392


Train DeepLearning model

In [13]:
train_input_list, train_label_list = toVectorFutureTrainFormat(train_files, DATA_DIR)
trainModel(train_input_list, train_label_list, num_epochs=8, batch_size=8)

Eval DeepLearning model

In [8]:
model = loadModel()

Evaluation of mulitple fingering for a single piece

In [ ]:
getScoresForDL(test_files, hmm_res_files, model)